In [34]:
import numpy as np
import pandas as pd
from scipy.stats import chi2_contingency

spdata = pd.read_pickle('SP.pkl')
mpdata = pd.read_pickle('MP.pkl')

In [35]:
# 标记 p < 0.05
def highlight_diff(rowdata):
    return ['background-color: darkgreen' if rowdata['p'] < 0.05 else '' for v in rowdata]

In [38]:
# 对照spdata, mpdata交集列显示统计信息
def columninfo(tag, spdata, mpdata):

    print('='*90)

    # 分项统计
    svalue = spdata.groupby(tag).size().sort_values(ascending=False)
    print('spdata:', svalue)
    print('-'*90)    
    mvalue = mpdata[mpdata['Record number recode']==1].groupby(tag).size().sort_values(ascending=False)
    print('mpdata:', mvalue)
    print('-'*90)

    # 项目列表
    snunique = spdata[tag].nunique()
    mnunique = mpdata[mpdata['Record number recode']==1][tag].nunique()
    print(f'spdata: {snunique}, mpdata: {mnunique}')
    sunique = spdata[tag].unique()
    munique = mpdata[mpdata['Record number recode']==1][tag].unique()
    index = list(set(sunique).intersection(set(munique)))
    print('difference:\n')
    sdiff = list(set(sunique).difference(set(munique)))
    print('spdata: ', sdiff)
    mdiff = list(set(munique).difference(set(sunique)))
    print('mpdata: ', mdiff)
    print('*'*90)

    # 同项目卡方检验，排序时用'0'，排序后换回0
    obs = np.array([svalue[index], mvalue[index]]).T

    obsframe = pd.DataFrame(index=index, columns=['svalue','mvalue','percent'])
    obsframe['svalue'] = obs[:,0]
    obsframe['mvalue'] = obs[:,1]
    obsframe['percent'] = round(obsframe['mvalue']/(obsframe['svalue']+obsframe['mvalue']), 3)
    print(obsframe.sort_values(by='percent', ascending=False))
    
    # 整体差异性
    print('p = ', round(chi2_contingency(obs)[1], 3))
    print('-'*90)

    # 组间两两比较
    result = pd.DataFrame()
    result_type1 = []
    result_percent1 = []
    result_type2 = []
    result_percent2 = []
    result_p = []
    for type1 in range(len(obs)):
        for type2 in range(type1+1, len(obs)):
            result_type1.append(index[type1])
            result_percent1.append(obsframe.loc[index[type1], 'percent'])
            result_type2.append(index[type2])
            result_percent2.append(obsframe.loc[index[type2], 'percent'])
            result_p.append(round(chi2_contingency(np.array([obs[type1], obs[type2]]))[1],3))
    result['type1'] = result_type1
    result['percent1'] = result_percent1
    result['type2'] = result_type2
    result['percent2'] = result_percent2
    result['p'] = result_p

    return result.style.apply(highlight_diff, axis=1).format({'p': '{:.3f}', 'percent1': '{:.3f}', 'percent2': '{:.3f}'})

In [39]:
# 项目交集
print('='*60)
intersection_columns = list(set(spdata.columns).intersection(set(mpdata.columns)))
print('intersection:\n', intersection_columns)
print('-'*60)
# 项目差集
spdata_difference_columns = list(set(spdata.columns).difference(set(mpdata.columns)))
print('spdata:\n', spdata_difference_columns)
print('-'*60)
mpdata_difference_columns = list(set(mpdata.columns).difference(set(spdata.columns)))
print('mpdata:\n', mpdata_difference_columns)
print('='*60)

intersection:
 ['RX Summ--Surg Prim Site (1998+)', 'Derived SEER Cmb Stg Grp (2016+)', 'SEER Combined Mets at DX-bone (2010+)', 'Sequence number', 'Derived AJCC T, 7th ed (2010-2015)', 'Derived SEER Combined N (2016+)', 'Patient ID', 'Radiation sequence with surgery', 'Derived AJCC N, 7th ed (2010-2015)', 'COD to site recode', 'SEER Combined Mets at DX-brain (2010+)', 'Race recode (W, B, AI, API)', 'Primary Site - labeled', 'Marital status at diagnosis', 'Site recode ICD-O-3/WHO 2008', 'Derived SEER Combined M (2016+)', 'Chemotherapy recode (yes, no/unk)', 'Survival months', 'SEER Combined Mets at DX-lung (2010+)', 'Derived AJCC M, 7th ed (2010-2015)', 'Derived SEER Combined T (2016+)', 'ER Status Recode Breast Cancer (1990+)', 'Derived HER2 Recode (2010+)', 'Age at diagnosis', 'Derived AJCC Stage Group, 7th ed (2010-2015)', 'PR Status Recode Breast Cancer (1990+)', 'Laterality', 'Radiation recode', 'Tumor Size Summary (2016+)', 'CS site-specific factor 7 (2004+ varying by schema)', 'Y

In [42]:
columninfo('Breast Subtype (2010+)', spdata, mpdata)

spdata: Breast Subtype (2010+)
HR+/HER2- (Luminal A)          212508
HR-/HER2- (Triple Negative)     33828
HR+/HER2+ (Luminal B)           33269
Unknown                         21166
HR-/HER2+ (HER2 enriched)       14376
dtype: int64
------------------------------------------------------------------------------------------
mpdata: Breast Subtype (2010+)
HR+/HER2- (Luminal A)          5741
HR-/HER2- (Triple Negative)     948
HR+/HER2+ (Luminal B)           696
Unknown                         604
HR-/HER2+ (HER2 enriched)       299
dtype: int64
------------------------------------------------------------------------------------------
spdata: 5, mpdata: 5
difference:

spdata:  []
mpdata:  []
******************************************************************************************
                             svalue  mvalue  percent
Unknown                       21166     604    0.028
HR-/HER2- (Triple Negative)   33828     948    0.027
HR+/HER2- (Luminal A)        212508    5741    0.026

,type1,percent1,type2,percent2,p
0,HR+/HER2+ (Luminal B),0.020,HR-/HER2+ (HER2 enriched),0.020,0.961
1,HR+/HER2+ (Luminal B),0.020,Unknown,0.028,0.000
2,HR+/HER2+ (Luminal B),0.020,HR+/HER2- (Luminal A),0.026,0.000
3,HR+/HER2+ (Luminal B),0.020,HR-/HER2- (Triple Negative),0.027,0.000
4,HR-/HER2+ (HER2 enriched),0.020,Unknown,0.028,0.000
5,HR-/HER2+ (HER2 enriched),0.020,HR+/HER2- (Luminal A),0.026,0.000
6,HR-/HER2+ (HER2 enriched),0.020,HR-/HER2- (Triple Negative),0.027,0.000
7,Unknown,0.028,HR+/HER2- (Luminal A),0.026,0.215
8,Unknown,0.028,HR-/HER2- (Triple Negative),0.027,0.752
9,HR+/HER2- (Luminal A),0.026,HR-/HER2- (Triple Negative),0.027,0.311
